In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import warnings
warnings.filterwarnings("ignore")

d:\NationalSecret\IE105\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv(r'D:\NationalSecret\IE105\data\all_ver2.csv')
data

,Payload,Label
0,id=2&nombre=Vino+Rioja&precio=39&cantidad=72&...,0
1,<login><username>john_doe</username><password>...,0
2,modo=registro&login=aronstam&password=373406N...,0
3,modo=entrar&login=delila&pwd=ipe$cacuana&remem...,0
4,<login><username>john_doe</username><password>...,0
...,...,...
66011,modo=registro&login=kimbroug&password=3m6noja4...,0
66012,<tbody onpointermove=alert(1)>XSS</tbody>,1
66013,<order><item>Phone</item><price>500</price></o...,0
66014,"<DIV STYLE=""background-image: url(javascript:a...",1


In [3]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data["Payload"].astype(str), data["Label"], test_size=0.2, stratify=data["Label"])

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)


class PayloadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
    

train_dataset = PayloadDataset(train_encodings, train_labels.tolist())
val_dataset = PayloadDataset(val_encodings, val_labels.tolist())

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

training_args = TrainingArguments(
    output_dir='./results',        
    num_train_epochs=3,  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,   
    warmup_steps=500,              
    weight_decay=0.01,           
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,                    
    args=training_args,   
    train_dataset=train_dataset, 
    eval_dataset=val_dataset 
)

trainer.train()
trainer.save_model('bert')
trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
